In [1]:
%pip install openpyxl

%pip install xlrd


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

# Load the Excel file
file_name = "sarani_dump.xlsx"

# Load the data from the 'room_master' sheet into a pandas DataFrame
try:
    # Specify the columns to load
    df = pd.read_excel(file_name, sheet_name="room_master", usecols=["room_no", "capacity"])
    print("Data successfully loaded into DataFrame.")
    
    # Rename the columns
    df.rename(columns={"room_no": "RoomNumber", "capacity": "Capacity"}, inplace=True)
    
    # Save the DataFrame to a new CSV file
    df.to_csv("classroom_master.csv", index=False)
    print("Filtered data successfully saved to 'classroom_master.csv'.")
    
except Exception as e:
    print(f"An error occurred: {e}")


Data successfully loaded into DataFrame.
Filtered data successfully saved to 'classroom_master.csv'.


In [3]:
# Load the Excel file
file_name = "sarani_dump.xlsx"

# Load the data from the 'room_master' sheet into a pandas DataFrame
try:
    # Specify the columns to load
    df = pd.read_excel(file_name, sheet_name="course_master", usecols=["code", "name"])
    print("Data successfully loaded into DataFrame.")
    
    # Rename the columns
    df.rename(columns={"code": "Code", "name": "Title"}, inplace=True)
    
    # Save the DataFrame to a new CSV file
    df.to_csv("course_master.csv", index=False)
    print("Filtered data successfully saved to 'course_master.csv'.")
    
except Exception as e:
    print(f"An error occurred: {e}")

Data successfully loaded into DataFrame.
Filtered data successfully saved to 'course_master.csv'.


In [4]:
# Load the Excel file
file_name = "T2-24-25-Enrollments-after-Add-Drop-Process-v3.xls"

# Load the data from the 'room_master' sheet into a pandas DataFrame
try:
    # Specify the columns to load
    df = pd.read_excel(file_name, sheet_name="Course Roster Report", usecols=["Section", "Email ID"])
    print("Data successfully loaded into DataFrame.")
    
    # Rename the columns
    df.rename(columns={"Section": "CourseCode"}, inplace=True)
    
    # Save the DataFrame to a new CSV file
    df.to_csv("enrollment_master.csv", index=False)
    print("Filtered data successfully saved to 'enrollment_master.csv'.")
    
except Exception as e:
    print(f"An error occurred: {e}")


Data successfully loaded into DataFrame.
Filtered data successfully saved to 'enrollment_master.csv'.
